In [14]:
import pandas as pd

# Load the CSV again with correct settings
df = pd.read_csv(
    '../data/911_Calls_for_Service.csv',
    sep='\t',
    on_bad_lines='skip',
    encoding='utf-8',
    engine='python'
)

# Convert datetime
df['callDateTime'] = pd.to_datetime(df['callDateTime'], errors='coerce')

# Drop empty column and null dates
df.drop(columns=['Unnamed: 19'], inplace=True)
df = df.dropna(subset=['callDateTime'])

# Create time features again
df['Hour'] = df['callDateTime'].dt.hour
df['DayOfWeek'] = df['callDateTime'].dt.day_name()
df['Month'] = df['callDateTime'].dt.month
df['Year'] = df['callDateTime'].dt.year

# Clean string fields
df['description'] = df['description'].str.strip()
df['priority'] = df['priority'].str.strip()
# Keep only rows with non-null ZIP codes
df_model = df.dropna(subset=['ZIPCode'])

# Use only most frequent ZIP codes (to avoid very rare classes)
top_zips = df_model['ZIPCode'].value_counts().nlargest(10).index
df_model = df_model[df_model['ZIPCode'].isin(top_zips)]

# Select useful features
features = df_model[['Hour', 'DayOfWeek', 'description', 'priority']]
target = df_model['ZIPCode']

# Encode categorical variables
from sklearn.preprocessing import LabelEncoder

le_day = LabelEncoder()
le_type = LabelEncoder()
le_priority = LabelEncoder()

features['DayOfWeek'] = le_day.fit_transform(features['DayOfWeek'])
features['description'] = le_type.fit_transform(features['description'])
features['priority'] = le_priority.fit_transform(features['priority'])


/var/folders/31/9x7_943517v2frgshb22nr_40000gn/T/ipykernel_21295/1897640000.py:46: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['DayOfWeek'] = le_day.fit_transform(features['DayOfWeek'])
/var/folders/31/9x7_943517v2frgshb22nr_40000gn/T/ipykernel_21295/1897640000.py:47: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  features['description'] = le_type.fit_transform(features['description'])
/var/folders/31/9x7_943517v2frgshb22nr_40000gn/T/ipykernel_21295/1897640000.py:48: SettingWithCopyWarning: 
A val

## Step 3: Train Model
Split data and train a Random Forest classifier.

In [15]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

X_train, X_test, y_train, y_test = train_test_split(features, target, test_size=0.3, random_state=42)

model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.15561707846386902
              precision    recall  f1-score   support

     21201.0       0.15      0.13      0.14     20289
     21202.0       0.18      0.19      0.19     21442
     21213.0       0.11      0.05      0.07     17154
     21215.0       0.15      0.34      0.21     28615
     21217.0       0.19      0.20      0.19     23588
     21218.0       0.14      0.13      0.13     22862
     21223.0       0.13      0.06      0.08     18440
     21224.0       0.15      0.16      0.16     22635
     21229.0       0.13      0.05      0.08     15436
     21230.0       0.16      0.11      0.13     20120

    accuracy                           0.16    210581
   macro avg       0.15      0.14      0.14    210581
weighted avg       0.15      0.16      0.14    210581



## Step 4: Simulate and Predict New Emergencies
We simulate future calls and predict ZIP codes.

In [16]:
# Predict ZIP for a new call: Monday, 14:00, NARCOTICS, High Priority
new_case = pd.DataFrame({
    'Hour': [14],
    'DayOfWeek': le_day.transform(['Monday']),
    'description': le_type.transform(['NARCOTICS']),
    'priority': le_priority.transform(['High'])
})

zip_prediction = model.predict(new_case)
print("Predicted ZIP code for dispatch:", zip_prediction[0])


Predicted ZIP code for dispatch: 21217.0


In [17]:
# Simulate 1000 future calls
import numpy as np

simulated_calls = pd.DataFrame({
    'Hour': np.random.randint(0, 24, 1000),
    'DayOfWeek': le_day.transform(np.random.choice(le_day.classes_, 1000)),
    'description': le_type.transform(np.random.choice(le_type.classes_, 1000)),
    'priority': le_priority.transform(np.random.choice(le_priority.classes_, 1000))
})

# Predict likely ZIPs for these calls
predicted_zips = model.predict(simulated_calls)

# Count frequency of ZIP predictions
from collections import Counter
zip_counts = Counter(predicted_zips)
top_zips = sorted(zip_counts.items(), key=lambda x: -x[1])[:5]

print("Top 5 ZIP codes to station drones:", top_zips)


Top 5 ZIP codes to station drones: [(np.float64(21215.0), 150), (np.float64(21202.0), 149), (np.float64(21224.0), 116), (np.float64(21217.0), 111), (np.float64(21223.0), 94)]


## Step 5: Assign Drones Based on Prediction

In [18]:
# Simulate 5 drones
drones = ['Drone_A', 'Drone_B', 'Drone_C', 'Drone_D', 'Drone_E']
assigned_zips = [zip_[0] for zip_ in top_zips]

drone_deployment = dict(zip(drones, assigned_zips))
print("Drone Deployment Plan:")
for drone, zone in drone_deployment.items():
    print(f"{drone} stationed at ZIP {zone}")


Drone Deployment Plan:
Drone_A stationed at ZIP 21215.0
Drone_B stationed at ZIP 21202.0
Drone_C stationed at ZIP 21224.0
Drone_D stationed at ZIP 21217.0
Drone_E stationed at ZIP 21223.0


## Step 6: Map Drone Locations

In [19]:
import folium

# Approximate centroids for top ZIP codes (Baltimore area)
zip_coords = {
    21201.0: (39.2950, -76.6215),
    21202.0: (39.2970, -76.6089),
    21213.0: (39.3125, -76.5848),
    21215.0: (39.3455, -76.6806),
    21217.0: (39.3117, -76.6371),
    21218.0: (39.3289, -76.6099),
    21223.0: (39.2807, -76.6464),
    21224.0: (39.2842, -76.5584),
    21229.0: (39.2862, -76.6926),
    21230.0: (39.2686, -76.6093),
}
# Create map centered on Baltimore
baltimore_center = [39.29, -76.61]
m = folium.Map(location=baltimore_center, zoom_start=12)

# Plot drone assignments from previous simulation
for drone, zip_code in drone_deployment.items():
    if zip_code in zip_coords:
        lat, lon = zip_coords[zip_code]
        folium.Marker(
            location=[lat, lon],
            popup=f"{drone} (ZIP {int(zip_code)})",
            icon=folium.Icon(color='red', icon='info-sign')
        ).add_to(m)

# Show map
m


## Step 7: Simulate Emergency Call Responses

In [20]:
import numpy as np

# Simulate 20 emergency calls
incoming_calls = pd.DataFrame({
    'Hour': np.random.randint(0, 24, 20),
    'DayOfWeek': le_day.transform(np.random.choice(le_day.classes_, 20)),
    'description': le_type.transform(np.random.choice(le_type.classes_, 20)),
    'priority': le_priority.transform(np.random.choice(le_priority.classes_, 20))
})

# Predict ZIPs for those calls
incoming_calls['PredictedZIP'] = model.predict(incoming_calls)

# Reverse the drone deployment mapping: ZIP → Drone
zip_to_drone = {v: k for k, v in drone_deployment.items()}

# Check which calls can be served instantly
def check_drone_response(zip_code):
    return zip_to_drone.get(zip_code, None)

incoming_calls['DroneAssigned'] = incoming_calls['PredictedZIP'].apply(check_drone_response)
incoming_calls['ResponseType'] = incoming_calls['DroneAssigned'].apply(
    lambda x: 'Immediate' if x is not None else 'Delayed'
)

# See the simulation result
incoming_calls[['PredictedZIP', 'DroneAssigned', 'ResponseType']].head(10)


,PredictedZIP,DroneAssigned,ResponseType
0,21202.0,Drone_B,Immediate
1,21201.0,None,Delayed
2,21217.0,Drone_D,Immediate
3,21215.0,Drone_A,Immediate
4,21217.0,Drone_D,Immediate
5,21217.0,Drone_D,Immediate
6,21202.0,Drone_B,Immediate
7,21201.0,None,Delayed
8,21230.0,None,Delayed
9,21215.0,Drone_A,Immediate


In [21]:
# Summary
print("Drone Response Simulation Summary:")
print(incoming_calls['ResponseType'].value_counts())


Drone Response Simulation Summary:
ResponseType
Immediate    10
Delayed      10
Name: count, dtype: int64
